In [1]:
from fred import Fred
from database import Database

import logging
log = logging.getLogger(__name__)

# import os
import time
import random
import pandas as pd

In [30]:
class FredETL:
    def __init__(self):
        api_key = 'b85eb30e34abc01edd5b64c248bb945b'
        self.fred = Fred(api_key)
        self.db = Database()
        self.metrics = None
        self.datasets = []

        self.gen_metrics()

    def extract(self):
        random_time = random.uniform(0.5, 1.0)

        for _topic,metrics in self.metrics.items():
            if _topic == 'state':
                state = 1
            else:
                state = 0
            
            for id,_desc in metrics.items():
                if state == 1:
                    topic = _desc[1]
                    desc = _desc[0]
                else:
                    topic = _topic
                    desc = _desc

                time.sleep(random_time)
                try:
                    data = pd.DataFrame(self.fred.get_series(id),columns=['metric'])
                    data = data.reset_index(drop=False).rename(columns={'index': 'metric_dt'})
                    data = (
                        data
                        .assign(metric_id=id)
                        .assign(metric_desc=desc)
                        .assign(state_ind=state)
                        .assign(topic=topic)
                    )
                    self.datasets.append(data)
                except:
                    # print(id)
                    pass

    def load(self):
        schema = 'base'
        table = 'fred_metrics_data'
        
        self.db.cur.execute(f'drop table if exists {schema}.{table}')
        self.db.conn.commit()
        
        for df in self.datasets:
            df.head()
            df.to_sql(table,self.db.engine,schema=schema,if_exists='append',index=False)
        
        self.db.close()

    def transform(self):
        None

    def gen_metrics(self):
        us_states = {
            'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
            'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
            'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL',
            'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
            'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
            'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
            'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
            'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
            'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE',
            'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
            'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
            'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
            'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
            'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
            'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
            'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
            'Wisconsin': 'WI', 'Wyoming': 'WY'
        }
        _state_metrics = {
            '{ST}UR': ('unemployment rate - {STATE}','unemployment'),
            'STTMINWG{ST}': ('State Minimum Wage Rate - {STATE}','earnings'),
            '{ST}UCSFRCONDOSMSAMID': ('Zillow Home Value Index (ZHVI) for All Homes Including Single Family Residences, Condos, and CO-OPs - {STATE}','housing'),
            '{ST}STHPI': ('All-Transactions House Price Index - {STATE}','housing'),
            '{ST}NQGSP': ('Gross Domestic Product: All Industry Total - {STATE}','gdp'),
            'MEHOINUS{ST}A672N': ('Real Median Household Income - {STATE}','earnings'),
            '{ST}PCPI': ('Per Capita Personal Income - {STATE}','earnings'),
            '{ST}PCE': ('Personal Consumption Expenditures: Total - {STATE}','consumption'),
            '{ST}ERET': ('Retail Trade Earnings - {STATE}','earnings'),
        }

        state_metrics = {}
        for state,st in us_states.items():
            for k,v in _state_metrics.items():
                state_metrics[k.format(ST=st)] = (v[0].format(STATE=state),v[1])
        self.metrics = {
            'unemployment': {
                'UNRATE': 'Unemployment Rate',
                # 'LNS14000006': 'Unemployment Rate - Black or African American',
                # 'CCSA': 'Continued Claims (Insured Unemployment)',
                # 'UEMPMEAN': 'Average Weeks Unemployed',
                # 'LNS14000002': 'Unemployment Rate - Women',
                # 'LNS14000009': 'Unemployment Rate - Hispanic or Latino',
                # 'LNS14000001': 'Unemployment Rate - Men',
                # 'LNS14000003': 'Unemployment Rate - White',
                # 'UEMP27OV': 'Number Unemployed for 27 Weeks & over',
            },
            # 'delinquency': {
            #     'DRCCLACBS': 'Delinquency Rate on Credit Card Loans, All Commercial Banks',
            #     'DRCLACBS': 'Delinquency Rate on Consumer Loans, All Commercial Banks',
            #     'DRALACBS': 'Delinquency Rate on All Loans, All Commercial Banks',
            #     'DRSFRMACBS': 'Delinquency Rate on Single-Family Residential Mortgages, Booked in Domestic Offices, All Commercial Banks',
            #     'DRBLACBS': 'Delinquency Rate on Business Loans, All Commercial Banks',
            #     'DRCRELEXFACBS': 'Delinquency Rate on Commercial Real Estate Loans (Excluding Farmland), Booked in Domestic Offices, All Commercial Banks',
            #     'DRCCLT100S': 'Delinquency Rate on Credit Card Loans, Banks Ranked 1st to 100th Largest in Size by Assets',
            #     'Q09084USQ507NNBR': 'Mortgage Delinquency Rates for United States',
            #     'DRSREACBS': 'Delinquency Rate on Loans Secured by Real Estate, All Commercial Banks',
            # },
            # 'charge off': {
            #     'CORCCACBS': 'Charge-Off Rate on Credit Card Loans, All Commercial Banks',
            #     'CORCCT100S': 'Charge-Off Rate on Credit Card Loans, Banks Ranked 1st to 100th Largest in Size by Assets',
            #     'CORCACBS': 'Charge-Off Rate on Consumer Loans, All Commercial Banks',
            #     'CORCREXFACBS': 'Charge-Off Rate on Commercial Real Estate Loans (Excluding Farmland), Booked in Domestic Offices, All Commercial Banks',
            #     'CORBLACBS': 'Charge-Off Rate on Business Loans, All Commercial Banks',
            #     'CORSREACBS': 'Charge-Off Rate on Loans Secured by Real Estate, All Commercial Banks',
            #     'COROCLACBS': 'Charge-Off Rate on Other Consumer Loans, All Commercial Banks',
            #     'CORCCT100S': 'Charge-Off Rate on Credit Card Loans, Banks Ranked 1st to 100th Largest in Size by Assets',
            # },
            # 'consumption': {
            #     'PCE': 'Personal Consumption Expenditures',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            # },
            # 'inflation': {
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            # },
            # 'housing': {
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            # },
            # 'gdp': {
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            # },
            # 'cpi': {
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            # },
            # 'm2': {
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            # },
            # 'earnings': {
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            #     '____': '____',
            # },
            'state': state_metrics,
        }

In [31]:
etl = FredETL()

In [32]:
etl.metrics

{'unemployment': {'UNRATE': 'Unemployment Rate'},
 'state': {'ALUR': ('unemployment rate - Alabama', 'unemployment'),
  'STTMINWGAL': ('State Minimum Wage Rate - Alabama', 'earnings'),
  'ALUCSFRCONDOSMSAMID': ('Zillow Home Value Index (ZHVI) for All Homes Including Single-Family Residences, Condos, and CO-OPs - Alabama',
   'housing'),
  'ALSTHPI': ('All-Transactions House Price Index - Alabama', 'housing'),
  'ALNQGSP': ('Gross Domestic Product: All Industry Total - Alabama', 'gdp'),
  'MEHOINUSALA672N': ('Real Median Household Income - Alabama', 'earnings'),
  'ALPCPI': ('Per Capita Personal Income - Alabama', 'earnings'),
  'ALPCE': ('Personal Consumption Expenditures: Total - Alabama',
   'consumption'),
  'ALERET': ('Retail Trade Earnings - Alabama', 'earnings'),
  'AKUR': ('unemployment rate - Alaska', 'unemployment'),
  'STTMINWGAK': ('State Minimum Wage Rate - Alaska', 'earnings'),
  'AKUCSFRCONDOSMSAMID': ('Zillow Home Value Index (ZHVI) for All Homes Including Single-Family 

In [33]:
etl.extract()

In [34]:
etl.load()

In [7]:
etl.datasets

[     metric_dt  metric metric_id        metric_desc  state_ind         topic
 0   1948-01-01     3.4    UNRATE  Unemployment Rate          0  unemployment
 1   1948-02-01     3.8    UNRATE  Unemployment Rate          0  unemployment
 2   1948-03-01     4.0    UNRATE  Unemployment Rate          0  unemployment
 3   1948-04-01     3.9    UNRATE  Unemployment Rate          0  unemployment
 4   1948-05-01     3.5    UNRATE  Unemployment Rate          0  unemployment
 ..         ...     ...       ...                ...        ...           ...
 900 2023-01-01     3.4    UNRATE  Unemployment Rate          0  unemployment
 901 2023-02-01     3.6    UNRATE  Unemployment Rate          0  unemployment
 902 2023-03-01     3.5    UNRATE  Unemployment Rate          0  unemployment
 903 2023-04-01     3.4    UNRATE  Unemployment Rate          0  unemployment
 904 2023-05-01     3.7    UNRATE  Unemployment Rate          0  unemployment
 
 [905 rows x 6 columns],
      metric_dt  metric    metric_id 